In [ ]:
import pandas as pd 
import numpy as np
import copy
train_data=pd.read_excel(r"E:\pxdata\use\5_5no1.xlsx",engine='openpyxl')
train_label= pd.read_excel(r"E:\pxdata\use\weizhi1.xlsx",engine='openpyxl')
#test_data=pd.read_excel(r"C:\Users\zhaoyuan\Desktop\数学建模\赛题\D\原始数据\Molecular_Descriptor.xlsx",engine='openpyxl',sheet_name='test')
#test_label= pd.read_excel(r"C:\Users\zhaoyuan\Desktop\数学建模\赛题\D\原始数据\ERα_activity.xlsx",engine='openpyxl',sheet_name='test')
train_data
#train_label

In [3]:
train_label

,x,y
0,5,5
1,5,5
2,5,5
3,5,5
4,5,5
5,5,5
6,5,5
7,5,5
8,5,5
9,5,5


In [12]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = (12.0, 12.0)
plt.rcParams['font.sans-serif'] = 'SimHei'
plt.title(train_label.columns[2]+"分布图")
sns.distplot(train_label.iloc[:,2])
plt.savefig("imgs/label_fig.png",dpi = 300)

IndexError: index 2 is out of bounds for axis 0 with size 2

In [4]:
train_label.iloc[:,0]==train_data.iloc[:,0]

0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16    False
17    False
18    False
19    False
20    False
21    False
22    False
dtype: bool

In [5]:
train_data[pd.isnull(train_data).T.any()]

,通道1到达时间ns,通道1幅度dB,通道1振铃计数,通道1持续时间us,通道1能量,通道2到达时间ns,通道2幅度dB,通道2振铃计数,通道2持续时间us,通道2能量,通道3到达时间ns,通道3幅度dB,通道3振铃计数,通道3持续时间us,通道3能量


In [6]:
#一列只有一个数值的变量 全为一个值
class_col_1 = []
#一列只有多个数值的变量
class_col = []
# class_col_2 = []
for col in range(train_data.shape[1]):
    num=0
    vals = set()
    for row in range(train_data.shape[0]):
        vals.add(train_data.iloc[row,col])
        if len(vals)>2:break
    if len(vals)==1:class_col_1.append(col)
    else: class_col.append(col)
#     else 
class_col

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14]

In [7]:
len(class_col)

14

In [8]:
# float64 = []
# int64 = []
# int64_num = []
# class_col_2 = []
#判断是否像[0,1,2]类型变量
def judge_int64(myset):
    last = -1
    for x in myset:
        if x-last!=1:
            return False
        last = x
    return True
#判断是否是离散或者连续变量
def get_int_float(mydata):
    myfloat64 = []
    myint64 = []
#     myint64_num = []
    for col in range(1,mydata.shape[1]):
        flag = True
        vals = set()
        for row in range(mydata.shape[0]):
            val=mydata.iloc[row,col]
            vals.add(val)
            if type(val)==np.float64:
                flag = False
                break
        #判断值是否连续   
        if flag and len(vals)<10 and judge_int64(vals):
            myint64.append(col)
         
        #         myint64_num.append(len(vals))
        else: myfloat64.append(col)
    return myint64,myfloat64
int64,float64 = get_int_float(train_data)
#     else 
len(int64)

0

In [9]:
# 异常值处理都需要对连续值进行
float_train_data = train_data.iloc[:,float64]
min_range= float_train_data.mean()-3*float_train_data.std()
max_range= float_train_data.mean()+3*float_train_data.std()
cols_mean = float_train_data.mean()
error_index = []
for row in range(float_train_data.shape[0]):
    for col in range(float_train_data.shape[1]):
        if float_train_data.iloc[row,col]<min_range[col] or float_train_data.iloc[row,col]>max_range[col]:
            error_index.append([row,col])
            float_train_data.iloc[row,col]=cols_mean[col]


In [10]:
len(error_index)

0

In [11]:
train_data.iloc[:,float64] = float_train_data

In [15]:
corr_limit = 0.8#0.8以上认为是强相关 
corr_matrix = train_data.iloc[:,1:].corr("pearson") 
corr_matrix.head()

,通道1幅度dB,通道1振铃计数,通道1持续时间us,通道1能量,通道2到达时间ns,通道2幅度dB,通道2振铃计数,通道2持续时间us,通道2能量,通道3到达时间ns,通道3幅度dB,通道3振铃计数,通道3持续时间us,通道3能量
通道1幅度dB,1.000000,0.751007,0.522150,0.823012,0.120896,0.623733,0.025522,0.004661,-0.087166,0.120897,NaN,-0.175961,0.313672,0.519770
通道1振铃计数,0.751007,1.000000,0.859958,0.649180,0.155450,0.319197,-0.377738,-0.420223,-0.500821,0.155451,NaN,-0.360482,0.364418,0.787839
通道1持续时间us,0.522150,0.859958,1.000000,0.306651,0.306940,-0.098304,-0.713389,-0.765177,-0.852229,0.306941,NaN,-0.533134,0.459980,0.906036
通道1能量,0.823012,0.649180,0.306651,1.000000,0.135415,0.734006,0.130522,0.177810,0.175155,0.135417,NaN,0.068477,0.060753,0.413992
通道2到达时间ns,0.120896,0.155450,0.306940,0.135415,1.000000,-0.061738,-0.133478,-0.321018,-0.304665,1.000000,NaN,-0.420840,0.397505,0.287629


In [16]:

corr_index = []
corr_num = []
corr_names = []
for row in range(corr_matrix.shape[0]):
    num=0
    names = []
    indexs = []
    for col in range(corr_matrix.shape[1]):
        if corr_matrix.iloc[row,col]>corr_limit and row!=col and col>row:
            indexs.append(corr_matrix.columns[col])
            
        if corr_matrix.iloc[row,col]>corr_limit:
            names.append(corr_matrix.columns[col])
            num+=1
    corr_num.append(num)
    corr_names.append(names)
    corr_index.append(indexs)
corr_index

[['通道1能量'],
 ['通道1持续时间us'],
 ['通道3能量'],
 [],
 ['通道3到达时间ns'],
 [],
 ['通道2持续时间us', '通道2能量'],
 ['通道2能量'],
 [],
 [],
 [],
 [],
 [],
 []]

In [23]:
# 即将删掉的列，强相关的自变量仅保留其一
del_cols = []
bianli_cols = []
arg_indexs = (-np.array(corr_num)).argsort()
for row in range(len(corr_index)):
    bianli_cols.append(corr_matrix.columns[row])
    for col in corr_index[row]:
        if col not in del_cols and col not in bianli_cols:
            
            、、del_cols.append(col)
del_cols
bianli_cols

['通道1幅度dB',
 '通道1振铃计数',
 '通道1持续时间us',
 '通道1能量',
 '通道2到达时间ns',
 '通道2幅度dB',
 '通道2振铃计数',
 '通道2持续时间us',
 '通道2能量',
 '通道3到达时间ns',
 '通道3幅度dB',
 '通道3振铃计数',
 '通道3持续时间us',
 '通道3能量']

In [24]:

import copy 
import mglearn
import matplotlib.pyplot as plt 

plt.rcParams['figure.figsize'] = (14.0, 14.0)
temp = copy.dee pcopy(corr_names[arg_indexs[0]])
temp = temp[len(temp)-5:-1]
temp.append(corr_matrix.columns[arg_indexs[0]])
grr = pd.plotting.scatter_matrix(train_data[temp],marker='o',c = train_label.iloc[:,2],hist_kwds={'bins':20})#,cmap=mglearn.cm3)
plt.savefig("imgs/第1题强相关变量之间的散点图1.png",dpi=300)

IndexError: single positional indexer is out-of-bounds

In [1]:
train_data.iloc[:,float64] = (train_data.iloc[:,float64]-train_data.iloc[:,float64].mean())/train_data.iloc[:,float64].std()
int64,float64 = get_int_float(ori_train_data)

ori_std = ori_train_data.std()
ori_mean = ori_train_data.mean()
#毒性方向 使1为最好值
train_label.iloc[:,[4,6]] = 1-train_label.iloc[:,[4,6]]
ori_train_data = (ori_train_data-ori_train_data.mean())/ori_train_data.std()
ori_train_data

NameError: name 'train_data' is not defined

In [10]:
import sklearn 
import torch
import torch.nn as nn
from timm.models.layers import trunc_normal_

ModuleNotFoundError: No module named 'torch'

In [7]:
from sklearn import linear_model
#导入模型，模型参数默认
LR = linear_model.LinearRegression()
#训练模型
LR.fit(train_data.iloc[:,:],train_label.iloc[:,1])
#预测模型LR.predict(X_test),此时输出类别数据
#打印截距
print('intercept_:' ,LR.intercept_)
#打印模型系数
print('coef_:', LR.coef_)
#LRcoef = train_data.iloc[:,1:].columns[(-abs(LR.coef_)).argsort()[:var_num]]
#LRcoef

intercept_: 9.050263319913622
coef_: [-4.60388955e-05  1.19673734e-01 -1.24400245e-02 -2.80367391e-05
 -1.42932311e-04  1.03081316e-04 -3.09395997e-02  5.19348561e-03
  1.33521248e-05 -5.27532115e-05 -5.70424401e-05 -7.11236625e-16
  6.20300835e-05  4.20094024e-05  6.72220045e-05]


In [5]:

# -*- coding：utf-8 -*-
# &Author  AnFany

# 利用神经网络拟合函数

# 生成训练数据
import numpy as np
import BPNN_DATA_Reg as bp
#  输入数据
xdata = train_data.iloc[:,:]

#  输出数据
#y1data = train_label.iloc[:,0]
#y2data = train_label.iloc[:,1]

#  两个输出数据和二为一，
ydata = train_label.iloc[:,:]

# 数据标准化
xdata = (xdata - np.mean(xdata, axis=0)) / np.std(xdata, axis=0)

# 最大与最小值
maxnum = np.max(ydata, axis=0, keepdims=True)

minnum = np.min(ydata, axis=0, keepdims=True)

nor m_ydata = (ydata - minnum) / (maxnum - minnum)

# 分为训练数据和测试数据
model_data = bp.divided(xdata, norm_ydata, percent=0.1)

# 训练的输入、输出
train_x_in = model_data[0]
train_y_out = model_data[1]

# 预测的输入、输出
pre_x_in = model_data[2]
pre_y_out = model_data[3]


# 引入AnFany以及TensorFlow方法

import AnFany_BPNN_Regression as An_Bpnn  
import TensorFlow_BPNN_Regression as Ten_Bpnn  


# AnFany方法
# # 开始训练数据
bpnn = An_Bpnn.BPNN(train_x_in, train_y_out, learn_rate=0.002, son_samples=50, iter_times=100000000, \
                    hidden_layer=[190, 190, 190], break_error=0.005)
bpnn_train = bpnn.train_adam()

# 训练结果展示
train_output = An_Bpnn.trans(bpnn.predict(train_x_in), minnum, maxnum)
An_Bpnn.figure(An_Bpnn.trans(train_y_out, minnum, maxnum), train_output, le='训练', width=4)

pre_output = An_Bpnn.trans(bpnn.predict(pre_x_in), minnum, maxnum)
An_Bpnn.figure(An_Bpnn.trans(pre_y_out, minnum, maxnum), pre_output, le='预测', width=2)

An_Bpnn.costfig(bpnn_train[2])


ModuleNotFoundError: No module named 'BPNN_DATA_Reg'

ModuleNotFoundError: No module named 'timm'

In [22]:
import numpy as np

def sigmod(z):
    h = 1./(1+np.exp(-z))
    return h
    
def de_sigmoid(z,h):
    return h*(1-h)
    
    
def relu(z):
    h = np.maximum(z, 0)
    return h
    
def de_relu(z,h):
    z[z <= 0] = 0
    z[z > 0] = 1.0
    return z
    
    
    
def no_active(z):
    h = z
    return h

def de_no_active(z,h):
    return np.ones(h.shape)
    
# o Nxc
# lab Nxc    
def loss_L2(o,lab):
    diff = lab-o
    sqrDiff = diff ** 2
    return 0.5*np.sum(sqrDiff)
    
def de_loss_L2(o,lab):
    return o-lab


def loss_CE(o,lab):    
    p = np.exp(o)/np.sum(np.exp(o),axis=1,keepdims=True)
    loss_ce = np.sum(-lab*np.log(p))
    return loss_ce

def de_loss_CE(o,lab):
    p = np.exp(o)/np.sum(np.exp(o),axis=1,keepdims=True)
    return p-lab

# dim_in:输入特征的维度
# list_num_hidden： 每层输出节点的数目
# list_act_funs： 每层的激活函数
# list_de_act_funs: 反向传播时的函数

def bulid_net(dim_in,list_num_hidden,
              list_act_funs,list_de_act_funs):
    layers=[]          
    
    # 逐层的进行网络构建
    for i in range(len(list_num_hidden)):
        layer = {}
        
        # 定义每一层的权重
        if i ==0:
            # layer["w"]= 0.2*np.random.randn(dim_in,list_num_hidden[i])-0.1 # 用sigmoid激活函数
            layer["w"]= 0.01*np.random.randn(dim_in,list_num_hidden[i])  # 用relu 激活函数
        else:
            # layer["w"]= 0.2*np.random.randn(list_num_hidden[i-1],list_num_hidden[i])-0.1 # 用sigmoid激活函数
            layer["w"]= 0.01*np.random.randn(list_num_hidden[i-1],list_num_hidden[i]) # 用relu 激活函数
        
        # 定义每一层的偏置
        layer["b"] = 0.1*np.ones([1,list_num_hidden[i]])
        layer["act_fun"]= list_act_funs[i]
        layer["de_act_fun"]= list_de_act_funs[i]
        layers.append(layer)
        
    return layers
    
    
# 返回每一层的输入
# 与最后一层的输出    
def fead_forward(datas,layers):
    input_layers = []
    input_acfun = []
    for i in range(len(layers)):
        layer = layers[i]
        if i ==0:
            inputs = datas
            z = np.dot(inputs,layer["w"]) + layer["b"]
            h = layer['act_fun'](z)
            input_layers.append(inputs)
            input_acfun.append(z)
        else:
            inputs = h
            z = np.dot(inputs,layer["w"])+ layer["b"]
            h = layer['act_fun'](z)
            input_layers.append(inputs)
            input_acfun.append(z)
    return input_layers,input_acfun,h


# 进行参数更新更新    
def updata_wb(datas,labs,layers, loss_fun,de_loss_fun,alpha=0.01):
    N,D = np.shape(datas)
    # 进行前馈操作
    inputs,input_acfun,output = fead_forward(datas,layers)
    # 计算 loss
    loss = loss_fun(output,labs)
    #从后向前计算
    deltas0 = de_loss_fun(output,labs)
    # 从后向前计算误差
    deltas =[]
    for i in range(len(layers)):
        index = -i-1
        if i ==0:
            h = output
            z = input_acfun[index]
            delta = deltas0*layers[index]["de_act_fun"](z,h)
        else:
            h = inputs[index+1]
            z = input_acfun[index]
            # print(layers[index]["de_act_fun"](z,h)[1])
            delta = np.dot(delta,layers[index+1]["w"].T)*layers[index]["de_act_fun"](z,h)
        
        deltas.insert(0,delta)
    
    # 利用误差 对每一层的权重进行修成
    for i in range(len(layers)):
        # 计算 dw 与 db
        dw = np.dot(inputs[i].T,deltas[i])
        db = np.sum(deltas[i],axis=0,keepdims=True)
        # 梯度下降
        layers[i]["w"] = layers[i]["w"] - alpha*dw
        layers[i]["b"] = layers[i]["b"] - alpha*db
        
    return layers,loss
    
    
def test_accuracy(datas,labs_true,layers):
    _,_,output = fead_forward(datas,layers)
    lab_det = np.argmax(output,axis=1)
    labs_true = np.argmax(labs_true,axis=1) 
    N_error = np.where(np.abs(labs_true-lab_det)>0)[0].shape[0]
   
    error_rate = N_error/np.shape(datas)[0]
    return error_rate

    
def load_dataset_iris(file_data,N_train):
    # 数据读取
    datas = np.loadtxt(file_data,dtype = np.float16, delimiter = ',',usecols=(0,1,2,3,4,5,6,7,8,9,10,11,12,13,14),encoding='utf-8')
    labs = np.loadtxt(file_data,dtype =np.float16, delimiter = ',',usecols=(15,16))
    N,D = np.shape(datas)
    N_test = N-N_train
    unqiue_labs = np.unique(labs).tolist()
    
    dic_str2index={}
    dic_index2str={}
    for i in range(len(unqiue_labs)):
        lab_str = unqiue_labs[i]
        dic_str2index[lab_str] =i
        dic_index2str[i]=lab_str
    
    labs_onehot = np.zeros([N,len(unqiue_labs)])
    for i in range(N):
        labs_onehot[i,dic_str2index[labs[i]]]=1
    
    perm = np.random.permutation(N)
    index_train = perm[:N_train]
    index_test = perm[N_train:]

    data_train = datas[index_train,:]
    lab_train_onehot = labs_onehot[index_train,:]

    data_test = datas[index_test,:]
    lab_test_onehot = labs_onehot[index_test]

    return data_train,lab_train_onehot,data_test,lab_test_onehot,dic_index2str
    
    
    
if __name__=="__main__":
    
    file_data = r'E:\pxdata\use\data_t1.txt'

    data_train,lab_train_onehot,data_test,lab_test_onehot,dic_index2str =load_dataset_iris(file_data,500)
    
    N,dim_in = np.shape(data_train)
    # 定义网络结构
    list_num_hidden=[15,15,30]
    list_act_funs =[relu,relu,no_active]
    list_de_act_funs=[de_relu,de_relu,de_no_active]
    
    # 定义损失函数
    loss_fun = loss_CE
    de_loss_fun=de_loss_CE
    
    # loss_fun = loss_L2
    # de_loss_fun=de_loss_L2
    
    layers = bulid_net(dim_in,list_num_hidden,
          list_act_funs,list_de_act_funs)
    
   
    # 进行训练
    n_epoch = 200
    batchsize =4    
    N_batch = N//batchsize
    for i in range(n_epoch):
        # 数据打乱
        rand_index  = np.random.permutation(N).tolist()
        # 每个batch 更新一下weight
        loss_sum =0
        for j in range(N_batch):
            index = rand_index[j*batchsize:(j+1)*batchsize]
            batch_datas = data_train[index]
            batch_labs = lab_train_onehot[index]
            layers,loss = updata_wb(batch_datas,batch_labs,layers,loss_fun,de_loss_fun,alpha=0.01)
            loss_sum = loss_sum+loss
            
        error = test_accuracy(data_train,lab_train_onehot,layers)
        print("epoch %d  error  %.2f%%  loss_all %.2f"%(i,error*100,loss_sum))
    
    #进行测试
    error = test_accuracy(data_test,lab_test_onehot,layers)
    print(error*100)   

IndexError: list index out of range

In [19]:
#-*- coding:utf-8 -*-
import numpy as np
from NN_BP import *

def load_mnist(file_data,file_lab):
    # 加载训练数据
    data = np.loadtxt(file_data)
    lab = np.loadtxt(file_lab)
    N,D = np.shape(data)
    
    # 构造 one-hot 标签
    lab_onehot = np.zeros([N,10])
    for i in range(N):
        id = int(lab[i,0])
        lab_onehot[i,id]=1
    data = (data.astype(np.float)/255.0)
    return data,lab_onehot
    

if __name__=="__main__":
    
    # 加载训练数据
    train_data,train_lab_onehot=load_mnist("E:\\pxdata\\use\\5_5no1.txt","E:\\pxdata\\use\\label1.txt")
    N,D = np.shape(train_data) 
    
    # 搭建网络
    # 定义网络结构
    list_num_hidden=[30,5,10]
    
    # list_act_funs =[sigmod,sigmod,no_active]
    # list_de_act_funs=[de_sigmoid,de_sigmoid,de_no_active]
    
    # # 定义损失函数
    # loss_fun = loss_L2
    # de_loss_fun=de_loss_L2
    
    list_act_funs =[relu,relu,no_active]
    list_de_act_funs=[de_relu,de_relu,de_no_active]
    # 定义损失函数
    loss_fun = loss_CE
    de_loss_fun=de_loss_CE
    
    layers = bulid_net(D,list_num_hidden,
          list_act_funs,list_de_act_funs)
          
    # 进行训练
    n_epoch = 50
    batchsize =20    
    N_batch = N//batchsize
    for i in range(n_epoch):
        # 数据打乱
        rand_index  = np.random.permutation(N).tolist()
        # 每个batch 更新一下weight
        loss_sum =0
        for j in range(N_batch):
            index = rand_index[j*batchsize:(j+1)*batchsize]
            batch_datas = train_data[index]
            batch_labs = train_lab_onehot[index]
            layers,loss = updata_wb(batch_datas,batch_labs,layers,loss_fun,de_loss_fun,alpha=0.001)
            # print("epoch %d  batch %d  loss %.2f"%(i,j,loss/batchsize))
            loss_sum = loss_sum+loss
            
        error = test_accuracy(train_data,train_lab_onehot,layers)
        print("epoch %d  error  %.2f%%  loss_all %.2f"%(i,error*100,loss_sum/(N_batch*batchsize)))
        
    np.save("model.npy",layers)    
    
    # 加载测试数据
    #test_data,test_lab_onehot=load_mnist("test_data.npy","test_lab.npy")
    #layers = np.load("model.npy",allow_pickle=True)
   
    #error = test_accuracy(test_data,test_lab_onehot,layers)
   # print("Accuarcy on Test Data %.2f %%"%((1-error)*100))
    

UnicodeDecodeError: 'gbk' codec can't decode byte 0xff in position 0: illegal multibyte sequence

In [30]:
import numpy as np

def sigmod(z):
    h = 1./(1+np.exp(-z))
    return h
    
def de_sigmoid(z,h):
    return h*(1-h)
    
    
def relu(z):
    h = np.maximum(z, 0)
    return h
    
def de_relu(z,h):
    z[z <= 0] = 0
    z[z > 0] = 1.0
    return z
    
    
    
def no_active(z):
    h = z
    return h

def de_no_active(z,h):
    return np.ones(h.shape)
    
# o Nxc
# lab Nxc    
def loss_L2(o,lab):
    diff = lab-o
    sqrDiff = diff ** 2
    return 0.5*np.sum(sqrDiff)
    
def de_loss_L2(o,lab):
    return o-lab


def loss_CE(o,lab):    
    p = np.exp(o)/np.sum(np.exp(o),axis=1,keepdims=True)
    loss_ce = np.sum(-lab*np.log(p))
    return loss_ce

def de_loss_CE(o,lab):
    p = np.exp(o)/np.sum(np.exp(o),axis=1,keepdims=True)
    return p-lab

# dim_in:输入特征的维度
# list_num_hidden： 每层输出节点的数目
# list_act_funs： 每层的激活函数
# list_de_act_funs: 反向传播时的函数

def bulid_net(dim_in,list_num_hidden,
              list_act_funs,list_de_act_funs):
    layers=[]          
    
    # 逐层的进行网络构建
    for i in range(len(list_num_hidden)):
        layer = {}
        
        # 定义每一层的权重
        if i ==0:
            # layer["w"]= 0.2*np.random.randn(dim_in,list_num_hidden[i])-0.1 # 用sigmoid激活函数
            layer["w"]= 0.01*np.random.randn(dim_in,list_num_hidden[i])  # 用relu 激活函数
        else:
            # layer["w"]= 0.2*np.random.randn(list_num_hidden[i-1],list_num_hidden[i])-0.1 # 用sigmoid激活函数
            layer["w"]= 0.01*np.random.randn(list_num_hidden[i-1],list_num_hidden[i]) # 用relu 激活函数
        
        # 定义每一层的偏置
        layer["b"] = 0.1*np.ones([1,list_num_hidden[i]])
        layer["act_fun"]= list_act_funs[i]
        layer["de_act_fun"]= list_de_act_funs[i]
        layers.append(layer)
        
    return layers
    
    
# 返回每一层的输入
# 与最后一层的输出    
def fead_forward(datas,layers):
    input_layers = []
    input_acfun = []
    for i in range(len(layers)):
        layer = layers[i]
        if i ==0:
            inputs = datas
            z = np.dot(inputs,layer["w"]) + layer["b"]
            h = layer['act_fun'](z)
            input_layers.append(inputs)
            input_acfun.append(z)
        else:
            inputs = h
            z = np.dot(inputs,layer["w"])+ layer["b"]
            h = layer['act_fun'](z)
            input_layers.append(inputs)
            input_acfun.append(z)
    return input_layers,input_acfun,h


# 进行参数更新更新    
def updata_wb(datas,labs,layers, loss_fun,de_loss_fun,alpha=0.01):
    N,D = np.shape(datas)
    # 进行前馈操作
    inputs,input_acfun,output = fead_forward(datas,layers)
    # 计算 loss
    loss = loss_fun(output,labs)
    #从后向前计算
    deltas0 = de_loss_fun(output,labs)
    # 从后向前计算误差
    deltas =[]
    for i in range(len(layers)):
        index = -i-1
        if i ==0:
            h = output
            z = input_acfun[index]
            delta = deltas0*layers[index]["de_act_fun"](z,h)
        else:
            h = inputs[index+1]
            z = input_acfun[index]
            # print(layers[index]["de_act_fun"](z,h)[1])
            delta = np.dot(delta,layers[index+1]["w"].T)*layers[index]["de_act_fun"](z,h)
        
        deltas.insert(0,delta)
    
    # 利用误差 对每一层的权重进行修成
    for i in range(len(layers)):
        # 计算 dw 与 db
        dw = np.dot(inputs[i].T,deltas[i])
        db = np.sum(deltas[i],axis=0,keepdims=True)
        # 梯度下降
        layers[i]["w"] = layers[i]["w"] - alpha*dw
        layers[i]["b"] = layers[i]["b"] - alpha*db
        
    return layers,loss
    
    
def test_accuracy(datas,labs_true,layers):
    _,_,output = fead_forward(datas,layers)
    lab_det = np.argmax(output,axis=1)
    labs_true = np.argmax(labs_true,axis=1) 
    N_error = np.where(np.abs(labs_true-lab_det)>0)[0].shape[0]
   
    error_rate = N_error/np.shape(datas)[0]
    return error_rate

    
def load_dataset_iris(file_data,N_train):
    # 数据读取
    datas = np.loadtxt(file_data,dtype = np.float, delimiter = ',',usecols=(0,1,2,3,4,5,6,7,8,9,10,11,12,13,14))
    labs = np.loadtxt(file_data,dtype = str, delimiter = ',',usecols=(15))
    N,D = np.shape(datas)
    N_test = N-N_train
    unqiue_labs = np.unique(labs).tolist()
    
    dic_str2index={}
    dic_index2str={}
    for i in range(len(unqiue_labs)):
        lab_str = unqiue_labs[i]
        dic_str2index[lab_str] =i
        dic_index2str[i]=lab_str
    
    labs_onehot = np.zeros([N,len(unqiue_labs)])
    for i in range(N):
        labs_onehot[i,dic_str2index[labs[i]]]=1
    
    perm = np.random.permutation(N)
    index_train = perm[:N_train]
    index_test = perm[N_train:]

    data_train = datas[index_train,:]
    lab_train_onehot = labs_onehot[index_train,:]

    data_test = datas[index_test,:]
    lab_test_onehot = labs_onehot[index_test]

    return data_train,lab_train_onehot,data_test,lab_test_onehot,dic_index2str
    
    
    
if __name__=="__main__":
    
    file_data = 'data_t1.data'

    data_train,lab_train_onehot,data_test,lab_test_onehot,dic_index2str =load_dataset_iris(file_data,10)
    
    N,dim_in = np.shape(data_train)
    # 定义网络结构
    list_num_hidden=[15,15,3]
    list_act_funs =[relu,relu,no_active]
    list_de_act_funs=[de_relu,de_relu,de_no_active]
    
    # 定义损失函数
    loss_fun = loss_CE
    de_loss_fun=de_loss_CE
    
    # loss_fun = loss_L2
    # de_loss_fun=de_loss_L2
    
    layers = bulid_net(dim_in,list_num_hidden,
          list_act_funs,list_de_act_funs)
    
   
    # 进行训练
    n_epoch = 50
    batchsize =4    
    N_batch = N//batchsize
    for i in range(n_epoch):
        # 数据打乱
        rand_index  = np.random.permutation(N).tolist()
        # 每个batch 更新一下weight
        loss_sum =0
        for j in range(N_batch):
            index = rand_index[j*batchsize:(j+1)*batchsize]
            batch_datas = data_train[index]
            batch_labs = lab_train_onehot[index]
            layers,loss = updata_wb(batch_datas,batch_labs,layers,loss_fun,de_loss_fun,alpha=0.01)
            loss_sum = loss_sum+loss
            
        error = test_accuracy(data_train,lab_train_onehot,layers)
        print("epoch %d  error  %.2f%%  loss_all %.2f"%(i,error*100,loss_sum))
    
    #进行测试
    error = test_accuracy(data_test,lab_test_onehot,layers)
    print(error*100)

<ipython-input-30-014c25cafec8>:150: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  datas = np.loadtxt(file_data,dtype = np.float, delimiter = ',',usecols=(0,1,2,3,4,5,6,7,8,9,10,11,12,13,14))
<ipython-input-30-014c25cafec8>:41: RuntimeWarning: overflow encountered in exp
  p = np.exp(o)/np.sum(np.exp(o),axis=1,keepdims=True)
<ipython-input-30-014c25cafec8>:41: RuntimeWarning: invalid value encountered in true_divide
  p = np.exp(o)/np.sum(np.exp(o),axis=1,keepdims=True)
<ipython-input-30-014c25cafec8>:42: RuntimeWarning: divide by zero encountered in log
  loss_ce = np.sum(-lab*np.log(p))


ValueError: operands could not be broadcast together with shapes (4,2) (4,3) 

In [32]:
import json
import random
import sys
import numpy as np

In [33]:
#### Define the quadratic and cross-entropy cost functions
class CrossEntropyCost(object):
 
    @staticmethod
    def fn(a, y):
        return np.sum(np.nan_to_num(-y*np.log(a)-(1-y)*np.log(1-a)))
 
    @staticmethod
    def delta(z, a, y):
        return (a-y)
 
#### Main Network class
class Network(object):
 
    def __init__(self, sizes, cost=CrossEntropyCost):
 
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.default_weight_initializer()
        self.cost=cost
 
    def default_weight_initializer(self):
 
        self.biases = [np.random.randn(y, 1) for y in self.sizes[1:]]
        self.weights = [np.random.randn(y, x)/np.sqrt(x)
                        for x, y in zip(self.sizes[:-1], self.sizes[1:])]
    def large_weight_initializer(self):
 
        self.biases = [np.random.randn(y, 1) for y in self.sizes[1:]]
        self.weights = [np.random.randn(y, x)
                        for x, y in zip(self.sizes[:-1], self.sizes[1:])]
    def feedforward(self, a):
        """Return the output of the network if ``a`` is input."""
        for b, w in zip(self.biases[:-1], self.weights[:-1]): # 前n-1层
            a = sigmoid(np.dot(w, a)+b)
 
        b = self.biases[-1]   # 最后一层
        w = self.weights[-1]
        a = np.dot(w, a)+b
        return a
 
    def SGD(self, training_data, epochs, mini_batch_size, eta,
            lmbda = 0.0,
            evaluation_data=None,
            monitor_evaluation_accuracy=False):  # 用随机梯度下降算法进行训练
 
        n = len(training_data)
 
        for j in xrange(epochs):
            random.shuffle(training_data)
            mini_batches = [training_data[k:k+mini_batch_size] for k in xrange(0, n, mini_batch_size)]
            
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta, lmbda, len(training_data))
            print ("Epoch %s training complete" % j)
            
            if monitor_evaluation_accuracy:
                print ("Accuracy on evaluation data: {} / {}".format(self.accuracy(evaluation_data), j))
         
    def update_mini_batch(self, mini_batch, eta, lmbda, n):
        """Update the network's weights and biases by applying gradient
        descent using backpropagation to a single mini batch.  The
        ``mini_batch`` is a list of tuples ``(x, y)``, ``eta`` is the
        learning rate, ``lmbda`` is the regularization parameter, and
        ``n`` is the total size of the training data set.
        """
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [(1-eta*(lmbda/n))*w-(eta/len(mini_batch))*nw
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb
                       for b, nb in zip(self.biases, nabla_b)]
 
    def backprop(self, x, y):
        """Return a tuple ``(nabla_b, nabla_w)`` representing the
        gradient for the cost function C_x.  ``nabla_b`` and
        ``nabla_w`` are layer-by-layer lists of numpy arrays, similar
        to ``self.biases`` and ``self.weights``."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # feedforward
        activation = x
        activations = [x] # list to store all the activations, layer by layer
        zs = [] # list to store all the z vectors, layer by layer
        for b, w in zip(self.biases[:-1], self.weights[:-1]):    # 正向传播 前n-1层
 
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
# 最后一层，不用非线性
        b = self.biases[-1]
        w = self.weights[-1]
        z = np.dot(w, activation)+b
        zs.append(z)
        activation = z
        activations.append(activation)
        # backward pass 反向传播
        delta = (self.cost).delta(zs[-1], activations[-1], y)   # 误差 Tj - Oj 
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())  #  (Tj - Oj) * O(j-1)
 
        for l in xrange(2, self.num_layers):
            z = zs[-l]    # w*a + b
            sp = sigmoid_prime(z)  # z * (1-z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp  # z*(1-z)*(Err*w) 隐藏层误差
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())  # Errj * Oi
        return (nabla_b, nabla_w)
 
    def accuracy(self, data):
 
        results = [(self.feedforward(x), y) for (x, y) in data]  
        alist=[np.sqrt((x[0][0]-y[0])**2+(x[1][0]-y[1])**2) for (x,y) in results]
 
        return np.mean(alist)
 
    def save(self, filename):
        """Save the neural network to the file ``filename``."""
        data = {"sizes": self.sizes,
                "weights": [w.tolist() for w in self.weights],
                "biases": [b.tolist() for b in self.biases],
                "cost": str(self.cost.__name__)}
        f = open(filename, "w")
        json.dump(data, f)
        f.close()
 
##Loading a Network
def load(filename):
    """Load a neural network from the file ``filename``.  Returns an
    instance of Network.
    """
    f = open(filename, "r")
    data = json.load(f)
    f.close()
    cost = getattr(sys.modules[__name__], data["cost"])
    net = Network(data["sizes"], cost=cost)
    net.weights = [np.array(w) for w in data["weights"]]
    net.biases = [np.array(b) for b in data["biases"]]
    return net
 
def sigmoid(z):
    """The sigmoid function.""" 
    return 1.0/(1.0+np.exp(-z))
  
def sigmoid_prime(z):
    """Derivative of the sigmoid function."""
    return sigmoid(z)*(1-sigmoid(z))

In [34]:
#coding: utf8
import my_datas_loader_1
import network_0

ModuleNotFoundError: No module named 'my_datas_loader_1'

In [35]:
import numpy as np
from math import sqrt
from sklearn.datasets import load_iris
from numpy import *

In [36]:
def bp_train(feature, label, n_hidden, maxCycle, alpha, n_output):
    """计算隐含层的输入
    input:  feature(mat): 特征
            label(mat):  标签
            n_hidden(int): 隐含层的节点数
            maxCycle(int):  最大的迭代次数
            alpha(float):  学习率
            n_output(int):  输出层的节点数
    output:  w0(mat):  输入层到隐含层之间的的权重
             b0(mat):  输入层到隐含层之间的偏置
             w1(mat):  隐含层到输出层之间的权重
             b1(mat):  隐含层到输出层之间的偏置
    """
    m, n = np.shape(feature)
    #1. 随机初始化参数（权重，偏置， 网络层结构， 激活函数）
    w0 = np.mat(np.random.rand(n, n_hidden))
    w0 = w0 * (8.0 * sqrt(6) / sqrt(n + n_hidden)) -\
         np.mat(np.ones((n, n_hidden))) * (4.0 * sqrt(6) / sqrt(n + n_hidden))
    b0 = np.mat(np.random.rand(1, n_hidden))
    b0 = b0 * (8.0 * sqrt(6) / sqrt(n + n_hidden)) -\
         np.mat(np.ones((1, n_hidden))) * (4.0 * sqrt(6) / sqrt(n + n_hidden))
    w1 = np.mat(np.random.rand(n_hidden, n_output))
    w1 = w1 * (8.0 * sqrt(6) / sqrt(n_hidden + n_output)) -\
         np.mat(np.ones((n_hidden, n_output))) * (4.0 * sqrt(6) / sqrt(n_hidden + n_output))
    b1 = np.mat(np.random.rand(1, n_output))
    b1 = b1 * (8.0 * sqrt(6) / sqrt(n_hidden + n_output)) -\
         np.mat(np.ones((1, n_output))) * (4.0 * sqrt(6) / sqrt(n_hidden + n_output))
    #2. 训练
    i = 0
    while i <= maxCycle:
        #信号正向传播
        #计算隐含层的输入
        hidden_input = hidden_in(feature, w0, b0)
        #计算隐含层的输出
        hidden_output = hidden_out(hidden_input)
        #计算输出层的输入
        output_in = predict_in(hidden_output, w1, b1)
        #计算输出层的输出
        output_out = predict_out(output_in)

        #误差的反向传播
        #隐含层到输出层之间的残差
        delta_output = -np.multiply((label - output_out), partial_sig(output_in))
        #输入层到隐含层之间的残差
        delta_hidden = np.multiply((delta_output * w1.T), partial_sig(hidden_input))

        #修正权重和偏置
        w1 = w1 - alpha * (hidden_output.T * delta_output)
        b1 = b1 - alpha * np.sum(delta_output, axis=0) * (1.0 / m)
        w0 = w0 - alpha * (feature.T * delta_hidden)
        b0 = b0 - alpha * np.sum(delta_hidden, axis=0) * (1.0 / m)
        if i % 100 == 0:
            print("\t------iter: ", i, ", cost: ", (1.0/2) * get_cost(get_predict(feature, w0, w1, b0, b1) - label))
        i += 1
    return w0, w1, b0, b1


## 计算隐含层的输入的hidden_in函数

In [37]:
def hidden_in(feature, w0, b0):
    """隐含层的输入
    input:  feature(mat): 特征
            w0(mat): 输入层到隐含层之间的权重
            b0(mat): 输入层到隐含层之间的偏置
    output:  hidden_in(mat): 隐含层的输入
    """
    m = np.shape(feature)[0]
    hidden_in = feature * w0
    for i in range(m):
        hidden_in[i, ] += b0
    return hidden_in

# 计算隐含层的输出的hidden_out函数

In [38]:
def hidden_out(hidden_in):
    """隐含层的输出
    input:  hidden_in(mat): 隐含层的输入
    output: hidden_output(mat): 隐含层的输出
    """
    hidden_output = sig(hidden_in)
    return hidden_output

# 计算输出层的输入的predict_in函数

In [39]:
def predict_in(hidden_out, w1, b1):
    """输出层的输入
    input:  hidden_out(mat): 隐含层的输出
            w1(mat): 隐含层到输出层之间的权重
            b1(mat): 隐含层到输出层之间的偏置
    output:  predict_in(mat): 输出层的输入
    """
    m = np.shape(hidden_out)[0]
    predict_in = hidden_out * w1
    for i in range(m):
        predict_in[i, ] += b1
    return predict_in

# 计算输出层的输出的predict_out函数

In [40]:
def predict_out(predict_in):
    """输出层的输出
    input:  predict_in(mat): 输出层的输入
    output:  result(mat): 输出层的输出
    """
    result = sig(predict_in)
    return result


# Sigmoid函数

In [45]:
def sig(x):
    """Sigmoid激活函数
    input:  x(mat/float): 自变量（矩阵或者任意实数）
    output: Sigmoid值（mat/float）: Sigmoid函数的值
    """
    return 1.0 / (1 + np.exp(-x))

# partial_sig函数

In [46]:
def partial_sig(x):
    m, n = np.shape(x)
    out = np.mat(np.zeros((m, n)))
    for i in range(m):
        for j in range(n):
            out[i, j] = sig(x[i, j]) * (1 - sig(x[i, j]))
    return out


# 计算损失函数值的get_cost函数

In [47]:
def get_cost(cost):
    m, n = np.shape(cost)
    cost_sum = 0.0
    for i in range(m):
        for j in range(n):
            cost_sum += cost[i, j] * cost[i, j]
    return cost_sum / m

# 计算错误率的err_rate函数

In [48]:
def err_rate(label, pre):
    m = np.shape(label)[0]
    err = 0.0
    for i in range(m):
        if label[i, 0] != pre[i, 0]:
            err += 1
    rate = err / m
    return rate

# 导入数据的load_data函数

In [49]:
def load_data():
    dataset_iris = load_iris()
    data_iris = dataset_iris['data']
    label_tmp = dataset_iris['target']
    m = len(label_tmp)
    n_class = len(set(label_tmp))
    print(label_tmp) 
    label_iris = np.mat(np.zeros((m, n_class)))
    for i in range(m):
        label_iris[i, label_tmp[i]] = 1
    print(label_iris)
    return data_iris, label_iris, n_class


In [50]:
#保存bp模型的save_model函数
def save_model(w0, w1, b0, b1):
    def write_file(filename, source):
        f = open(filename, "w")
        m, n = np.shape(source)
        for i in range(m):
            tmp = []
            for j in range(n):
                tmp.append(str(source[i, j]))
            f.write("\t".join(tmp) + "\n")
        f.close()
    write_file("weight_w0", w0)
    write_file("weight_w1", w1)
    write_file("weight_b0", b0)
    write_file("weight_b1", b1)

#对测试样本进行预测的get_predict函数
def get_predict(feature, w0, w1, b0, b1):
    return predict_out(predict_in(hidden_out(hidden_in(feature, w0, b0)), w1, b1))


# 训练BP模型的主函数

In [55]:
#训练BP模型的主函数
if __name__ == "__main__":
    #1.导入数据
    print("----------1. load Data-------------")
    feature, label, n_class = load_data()
    #2.训练模型
    print("----------2. training--------------")
    w0, w1, b0, b1 = bp_train(feature, label, 30, 1000, 0.01, n_class)
    #3.保存模型
    print("----------3. save model-------------")
    save_model(w0, w1, b0, b1)
    #4.得到最终的预测结果
    print("----------4. get prediction----------")
    result = get_predict(feature, w0, w1, b0, b1)
    print("训练准确性为： ", (1 - err_rate(np.argmax(label, axis=1), np.argmax(result, axis=1))))
 

----------1. load Data-------------
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]
[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.

TypeError: cannot unpack non-iterable NoneType object

In [7]:
#coding: utf-8
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np
from math import sqrt
#from sklearn.datasets import load_iris
from numpy import *

def standardization(data_class, handel_index):
    """
    标准化
    -数据无空值
    -数据经过 parse 方法格式转换
    :param data_class:
    :param handel_index:
    :return:
    """
    mean_list = {}
    std_list = {}
    for j in handel_index:
        col = []
        for i in range(len(data_class.data)):
            col.append(data_class.data[i][j])
        mean_list[j] = np.mean(col, axis=0)
        std_list[j] = np.std(col, axis=0)
        col = (col - mean_list[j]) / std_list[j]
        for i in range(len(data_class.data)):
            data_class.data[i][j] = col[i]

    data_class.standard_mean = mean_list
    data_class.standard_std = std_list
    return data_class

#BP神经网络模型的训练
def bp_train(feature, label, n_hidden, maxCycle, alpha, n_output):
    """计算隐含层的输入
        input:  feature(mat): 特征
                label(mat):  标签
                n_hidden(int): 隐含层的节点数
                maxCycle(int):  最大的迭代次数
                alpha(float):  学习率
                n_output(int):  输出层的节点数
        output:  w0(mat):  输入层到隐含层之间的的权重
                 b0(mat):  输入层到隐含层之间的偏置
                 w1(mat):  隐含层到输出层之间的权重
                 b1(mat):  隐含层到输出层之间的偏置
    """
    m, n = np.shape(feature)
    #1. 随机初始化参数（权重，偏置， 网络层结构， 激活函数）
    w0 = np.mat(np.random.rand(n, n_hidden))
    w0 = w0 * (8.0 * sqrt(6) / sqrt(n + n_hidden)) -\
         np.mat(np.ones((n, n_hidden))) * (4.0 * sqrt(6) / sqrt(n + n_hidden))
    b0 = np.mat(np.random.rand(1, n_hidden))
    b0 = b0 * (8.0 * sqrt(6) / sqrt(n + n_hidden)) -\
         np.mat(np.ones((1, n_hidden))) * (4.0 * sqrt(6) / sqrt(n + n_hidden))
    w1 = np.mat(np.random.rand(n_hidden, n_output))
    w1 = w1 * (8.0 * sqrt(6) / sqrt(n_hidden + n_output)) -\
         np.mat(np.ones((n_hidden, n_output))) * (4.0 * sqrt(6) / sqrt(n_hidden + n_output))
    b1 = np.mat(np.random.rand(1, n_output))
    b1 = b1 * (8.0 * sqrt(6) / sqrt(n_hidden + n_output)) -\
         np.mat(np.ones((1, n_output))) * (4.0 * sqrt(6) / sqrt(n_hidden + n_output))

    #2. 训练
    i = 0
    while i <= maxCycle:
        #信号正向传播
        #计算隐含层的输入
        hidden_input = hidden_in(feature, w0, b0)
        #计算隐含层的输出
        hidden_output = hidden_out(hidden_input)
        #计算输出层的输入
        output_in = predict_in(hidden_output, w1, b1)
        #计算输出层的输出
        output_out = predict_out(output_in)

        #误差的反向传播
        #隐含层到输出层之间的残差
        delta_output = -np.multiply((label - output_out), partial_sig(output_in))
        #输入层到隐含层之间的残差
        delta_hidden = np.multiply((delta_output * w1.T), partial_sig(hidden_input))

        #修正权重和偏置
        w1 = w1 - alpha * (hidden_output.T * delta_output)
        b1 = b1 - alpha * np.sum(delta_output, axis=0) * (1.0 / m)
        w0 = w0 - alpha * (feature.T * delta_hidden) 
        b0 = b0 - alpha * np.sum(delta_hidden, axis=0) * (1.0 / m)
        if i % 100 == 0:
            print("\t------iter: ", i, ", cost: ", (1.0/2) * get_cost(get_predict(feature, w0, w1, b0, b1) - label))
        i += 1
    return w0, w1, b0, b1

#计算隐含层的输入的hidden_in函数
def hidden_in(feature, w0, b0):
    """隐含层的输入
    input:  feature(mat): 特征
            w0(mat): 输入层到隐含层之间的权重
            b0(mat): 输入层到隐含层之间的偏置
    output:  hidden_in(mat): 隐含层的输入
    """
    m = np.shape(feature)[0]
    hidden_in = feature * w0
    for i in range(m):
        hidden_in[i, ] += b0
    return hidden_in

#计算隐含层的输出的hidden_out函数
def hidden_out(hidden_in):
    """隐含层的输出
    input:  hidden_in(mat): 隐含层的输入
    output: hidden_output(mat): 隐含层的输出
    """
    hidden_output = sig(hidden_in)
    return hidden_output

#计算输出层的输入的predict_in函数
def predict_in(hidden_out, w1, b1):
    """输出层的输入
    input:  hidden_out(mat): 隐含层的输出
            w1(mat): 隐含层到输出层之间的权重
            b1(mat): 隐含层到输出层之间的偏置
    output:  predict_in(mat): 输出层的输入
    """
    m = np.shape(hidden_out)[0]
    predict_in = hidden_out * w1
    for i in range(m):
        predict_in[i, ] += b1
    return predict_in

#计算输出层的输出的predict_out函数
def predict_out(predict_in):
    """输出层的输出
    input:  predict_in(mat): 输出层的输入
    output:  result(mat): 输出层的输出
    """
    result = sig(predict_in)
    return result

#Sigmoid函数
# def sig(x):
#     """Sigmoid激活函数
#     input:  x(mat/float): 自变量（矩阵或者任意实数）
#     output: Sigmoid值（mat/float）: Sigmoid函数的值
#     """
#     return 1/(1+np.exp(-x))
def sig(x):
    x_ravel = x.ravel()  # 将numpy数组展平
    length = len(x_ravel)
    y = []
    for index in range(length):
        if x_ravel[index].any() >= 0:
            y.append(1.0 / (1 + np.exp(-x_ravel[index])))
        else:
            y.append(np.exp(x_ravel[index]) / (np.exp(x_ravel[index]) + 1))
    return np.array(y).reshape(x.shape)

# def sig(x):
#     #from numpy import exp
#     #return 1.0/(1+exp(-inX))
#     #优化
#     if x.any()>=0:
#         return 1.0/(1+exp(-x))
#     else:
#         return exp(x)/(1+exp(x))
#partial_sig函数 
def partial_sig(x):
    m, n = np.shape(x)
    out = np.mat(np.zeros((m, n)))
    for i in range(m):
        for j in range(n):
            out[i, j] = sig(x[i, j]) * (1 - sig(x[i, j]))
    return out

#计算损失函数值的get_cost函数
def get_cost(cost):
    m, n = np.shape(cost)
    cost_sum = 0.0
    for i in range(m):
        for j in range(n):
            cost_sum += cost[i, j] * cost[i, j]
    return cost_sum / m

#计算错误率的err_rate函数
def err_rate(label, pre):
    m = np.shape(label)[0]
    err = 0.0
    for i in range(m):
        if label[i, 0] != pre[i, 0]:
            err += 1
    rate = err / m
    return rate


#导入数据的load_data函数
# def load_data(filename):
#     """导入训练数据
#     input：  filename(string): 文件名
#     output:  feature_name(mat): 特征
#             label_data(mat): 标签
#             n_class(int): 类别的个数
#     """                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          
#     #1.获取特征
#     f = open(filename) #也可以用上下文管理器进行管理文件的打开和关闭 with open(filename) as f:这样就不用f.close进行关闭
#     feature_data = []
#     label_tmp = []
#     for line in f.readlines():
#         feature_tmp = []
#         lines = line.strip().split(",")
#         for i in range(len(lines) - 1):
#             feature_tmp.append(float(lines[i]))
#         label_tmp.append(int(lines[-1]))  
#         feature_data.append(feature_tmp)
#     f.close()
def load_data(filename_d, filename_p):
    """导入训练数据
    input：  filename(string): 文件名
    output:  feature_name(mat): 特征
            label_data(mat): 标签
            n_class(int): 类别的个数
    """                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          
    #1.获取特征
    f = open(filename_d) #也可以用上下文管理器进行管理文件的打开和关闭 with open(filename) as f:这样就不用f.close进行关闭
    feature_data = []
    
    for line in f.readlines():
        feature_tmp = []
        lines = line.strip().split(",")
        for i in range(len(lines) ):
            feature_tmp.append(float(lines[i]))
        #label_tmp.append(int(lines[-1]))  
        feature_data.append(feature_tmp)
    p = open(filename_p) #也可以用上下文管理器进行管理文件的打开和关闭 with open(filename) as f:这样就不用f.close进行关闭
    label_data = []
    
    for line in p.readlines():
        label_tmp = []
        lines = line.strip().split(",")
        for i in range(len(lines) ):
            label_tmp.append(float(lines[i]))
        #label_tmp.append(int(lines[-1]))  
        label_data.append(label_tmp)    
    f.close()
    p.close()
    #2.获取标签
    m = len(label_tmp)
#     #2.获取标签
#     m = len(label_tmp)
#     n_class = len(set(label_tmp))
#     #label_data = np.mat(np.zeros((m , n_class)))
#     label_data = label_tmp
# #     for i in range(m):
# #         label_data[i, label_tmp[i]] = 1
    return np.mat(feature_data), np.mat(label_data), m

# def load_data():
#     dataset_iris = load_iris()
#     data_iris = dataset_iris['data']
#     label_tmp = dataset_iris['target']
#     m = len(label_tmp)
#     n_class = len(set(label_tmp))
#     print(label_tmp)
#     label_iris = np.mat(np.zeros((m, n_class)))
#     for i in range(m):
#         label_iris[i, label_tmp[i]] = 1
#     print(label_iris)
#     return data_iris, label_iris, n_class


#保存bp模型的save_model函数
def save_model(w0, w1, b0, b1):
    def write_file(filename, source):
        f = open(filename, "w")
        m, n = np.shape(source)
        for i in range(m):
            tmp = []
            for j in range(n):
                tmp.append(str(source[i, j]))
            f.write("\t".join(tmp) + "\n")
        f.close()
    write_file("weight_w0", w0)
    write_file("weight_w1", w1)
    write_file("weight_b0", b0)
    write_file("weight_b1", b1)

#对测试样本进行预测的get_predict函数
def get_predict(feature, w0, w1, b0, b1):
    return predict_out(predict_in(hidden_out(hidden_in(feature, w0, b0)), w1, b1))


#训练BP模型的主函数
if __name__ == "__main__":
    #1.导入数据
    print("----------1. load Data-------------")
#     filename='data_t1x.csv'
#     feature, label, n_class = load_data(filename)
#     feature1 =  (feature-feature.mean())/feature.std()
    filename_d='dataframe.csv'
    filename_p='location.csv'
    feature, label, n_class = load_data(filename_d, filename_p)
    feature_pd = pd.DataFrame(feature)
    label_pd = pd.DataFrame(label)
    min_max_scaler = MinMaxScaler()
    min_max_scaler.fit(feature_pd)
    x_train = min_max_scaler.transform(feature_pd)

    min_max_scaler.fit(label_pd)
    y_train = min_max_scaler.transform(label_pd)

# 验证集归一化
#     min_max_scaler.fit(x_valid_pd)
#     x_valid = min_max_scaler.transform(x_valid_pd)

#     min_max_scaler.fit(y_valid_pd)
#     y_valid = min_max_scaler.transform(y_valid_pd)
    #feature1 =  (feature-feature.mean())/feature.std()
    #feature2 = (feature-feature.min())/feature.max()-feature.min()
    
    print("feature: ",x_train)
    
    #2.训练模型
    print("----------2. training--------------")
    w0, w1, b0, b1 = bp_train(x_train, y_train,9, 5000, 0.01, n_class)
    #3.保存模型
    print("----------3. save model-------------")
    save_model(w0, w1, b0, b1)
    #4.得到最终的预测结果
    print("----------4. get prediction----------")
    y_new = get_predict(x_train, w0, w1, b0, b1)
    #min_max_scaler.fit(result)
    y_new = min_max_scaler.inverse_transform(y_new)
    print(y_new)
   # print(label)
    #print("训练准确性为： ", (1 - err_rate(np.argmax(label, axis=1), np.argmax(result, axis=1))))
   # print(result)



----------1. load Data-------------
feature:  [[0.05261478 0.87755822 0.49340511 0.78464419 0.67261779 0.05261549
  0.71827411 0.17403481 0.44169611 0.70079513 0.05261558 0.62991041
  0.41286111 0.44933921 0.70313799]
 [0.19120343 0.         0.01000046 0.01498127 0.04939048 0.19064568
  0.6998731  0.1410462  0.41813899 0.55569561 0.19064611 0.65816678
  0.32134958 0.43281938 0.59809266]
 [0.31898005 0.99894143 0.69235837 0.96629213 0.70557917 0.31898101
  0.93432741 0.26498258 0.58892815 0.74145385 0.31898094 0.98414886
  0.6256806  0.6123348  0.82388308]
 [0.84685413 0.93013409 0.6430282  0.8071161  0.70902865 0.84685423
  0.88261421 0.29364703 0.57714959 0.88675398 0.84685416 0.81977946
  0.64670587 0.54185022 0.80960964]
 [1.         0.85956246 0.49658798 0.80337079 0.79000732 1.
  0.81376904 0.21581378 0.49234393 0.74808995 1.         0.75740868
  0.48673251 0.51101322 0.83074777]
 [0.04074337 0.85673959 0.54101369 0.72846442 0.72720353 0.04074419
  0.77252538 0.24502415 0.47585395

	------iter:  100 , cost:  0.07118729484274904
	------iter:  200 , cost:  0.06259744005241144
	------iter:  300 , cost:  0.05796540802127205
	------iter:  400 , cost:  0.054662634182427755
	------iter:  500 , cost:  0.05208577301374326
	------iter:  600 , cost:  0.04993826563606494
	------iter:  700 , cost:  0.04805936153441346
	------iter:  800 , cost:  0.04636884976521117
	------iter:  900 , cost:  0.04483019846927361
	------iter:  1000 , cost:  0.04342792288396499
	------iter:  1100 , cost:  0.042154649432645894
	------iter:  1200 , cost:  0.041004186779979634
	------iter:  1300 , cost:  0.03996852208614446
	------iter:  1400 , cost:  0.03903733683821998
	------iter:  1500 , cost:  0.038198815906957964
	------iter:  1600 , cost:  0.03744078724156249
	------iter:  1700 , cost:  0.03675164675996705
	------iter:  1800 , cost:  0.036120900757779593
	------iter:  1900 , cost:  0.03553937426255149
	------iter:  2000 , cost:  0.0349992022704625
	------iter:  2100 , cost:  0.034493710250000

In [6]:
#coding: utf-8
import numpy as np
#import sys
#sys.path.append("F:\Machine_Learning_Algorithm\BP_Neural_Network")
#from Training_BP_NN import get_predict

#生成测试样本的generate_data函数
def generate_data():
    """在[-4.5, 4.5]之间随机生成20000组点"""
    #1. 随机生成数据点
    data = np.mat(np.zeros((200, 4)))
    m = np.shape(data)[0]
    x = np.mat(np.random.rand(200, 4))
    for i in range(m):
        data[i, 0] = x[i, 0] * 4 + 4
        data[i, 1] = x[i, 1] * 3 + 2
        data[i, 2] = x[i, 2] * 6 + 1
        data[i, 3] = x[i, 3] * 3

    #2. 将数据点保存到文件“test_data”中
    f = open("test_data", "w")
    m, n = np.shape(data)
    for i in range(m):
        tmp = []
        for j in range(n):
            tmp.append(str(data[i, j]))
        f.write("\t".join(tmp) + "\n")
    f.close()

#导入测试数据的load_data函数
def load_data(filename):
    f = open(filename)
    feature_data = []
    for line in f.readlines():
        feature_tmp = []
        lines = line.strip().split(",")
        for i in range(len(lines)):
            feature_tmp.append(float(lines[i]))
        feature_data.append(feature_tmp)
    f.close()
    return np.mat(feature_data)

#导入BP神经网络模型的load_model函数
def load_model(file_w0, file_w1, file_b0, file_b1):
    def get_model(file_name):
        f = open(file_name)
        model = []
        for line in f.readlines():
            lines = line.strip().split("\t")
            model_tmp = []
            for x in lines:
                model_tmp.append(float(x.strip()))
            model.append(model_tmp)
        f.close()
        return np.mat(model)
    w0 = get_model(file_w0)
    w1 = get_model(file_w1)
    b0 = get_model(file_b0)
    b1 = get_model(file_b1)
    return w0, w1, b0, b1

#保存最终的预测结果的save_predict函数
def save_predict(filename, pre):
    f = open(filename, "w")
    m = np.shape(pre)[0]
    result = []
    for i in range(m):
        result.append(str(pre[i, 0]))
    f.write("\n".join(result))
    f.close()



#对新数据进行预测的主函数
if __name__ == "__main__":
    #generate_data()
    #1. 导入数据
    print("-------------1. load data--------------")
    dataTest = load_data("dataframe.csv")
    #2. 导入BP神经网络模型
    print("-------------2. load model--------------")
    w0, w1, b0, b1 = load_model("weight_w0", "weight_w1", "weight_b0", "weight_b1")
    #3. 得到最终的预测值
    print("-------------3. get prediction------------")
    result = get_predict(dataTest, w0, w1, b0, b1)
    #. 保存最终的预测结果
    print("-------------4. save result---------------")
    pre = np.argmax(result, axis=1)
    save_predict("result", pre)

-------------1. load data--------------
-------------2. load model--------------
-------------3. get prediction------------
-------------4. save result---------------


<ipython-input-4-52978a379ca5>:149: RuntimeWarning: overflow encountered in exp
  y.append(1.0 / (1 + np.exp(-x_ravel[index])))


TypeError: cannot unpack non-iterable int object

In [1]:
import numpy as np

def sigmod(z):
    h = 1./(1+np.exp(-z))
    return h
    
def de_sigmoid(z,h):
    return h*(1-h)
    
    
def relu(z):
    h = np.maximum(z, 0)
    return h
    
def de_relu(z,h):
    z[z <= 0] = 0
    z[z > 0] = 1.0
    return z
    
    
    
def no_active(z):
    h = z
    return h

def de_no_active(z,h):
    return np.ones(h.shape)
    
# o Nxc
# lab Nxc    
def loss_L2(o,lab):
    diff = lab-o
    sqrDiff = diff ** 2
    return 0.5*np.sum(sqrDiff)
    
def de_loss_L2(o,lab):
    return o-lab


def loss_CE(o,lab):    
    p = np.exp(o)/np.sum(np.exp(o),axis=1,keepdims=True)
    loss_ce = np.sum(-lab*np.log(p))
    return loss_ce

def de_loss_CE(o,lab):
    p = np.exp(o)/np.sum(np.exp(o),axis=1,keepdims=True)
    return p-lab

# dim_in:输入特征的维度
# list_num_hidden： 每层输出节点的数目
# list_act_funs： 每层的激活函数
# list_de_act_funs: 反向传播时的函数

def bulid_net(dim_in,list_num_hidden,
              list_act_funs,list_de_act_funs):
    layers=[]          
    
    # 逐层的进行网络构建
    for i in range(len(list_num_hidden)):
        layer = {}
        
        # 定义每一层的权重
        if i ==0:
            # layer["w"]= 0.2*np.random.randn(dim_in,list_num_hidden[i])-0.1 # 用sigmoid激活函数
            layer["w"]= 0.01*np.random.randn(dim_in,list_num_hidden[i])  # 用relu 激活函数
        else:
            # layer["w"]= 0.2*np.random.randn(list_num_hidden[i-1],list_num_hidden[i])-0.1 # 用sigmoid激活函数
            layer["w"]= 0.01*np.random.randn(list_num_hidden[i-1],list_num_hidden[i]) # 用relu 激活函数
        
        # 定义每一层的偏置
        layer["b"] = 0.1*np.ones([1,list_num_hidden[i]])
        layer["act_fun"]= list_act_funs[i]
        layer["de_act_fun"]= list_de_act_funs[i]
        layers.append(layer)
        
    return layers
    
    
# 返回每一层的输入
# 与最后一层的输出    
def fead_forward(datas,layers):
    input_layers = []
    input_acfun = []
    for i in range(len(layers)):
        layer = layers[i]
        if i ==0:
            inputs = datas
            z = np.dot(inputs,layer["w"]) + layer["b"]
            h = layer['act_fun'](z)
            input_layers.append(inputs)
            input_acfun.append(z)
        else:
            inputs = h
            z = np.dot(inputs,layer["w"])+ layer["b"]
            h = layer['act_fun'](z)
            input_layers.append(inputs)
            input_acfun.append(z)
    return input_layers,input_acfun,h


# 进行参数更新更新    
def updata_wb(datas,labs,layers, loss_fun,de_loss_fun,alpha=0.01):
    N,D = np.shape(datas)
    # 进行前馈操作
    inputs,input_acfun,output = fead_forward(datas,layers)
    # 计算 loss
    loss = loss_fun(output,labs)
    #从后向前计算
    deltas0 = de_loss_fun(output,labs)
    # 从后向前计算误差
    deltas =[]
    for i in range(len(layers)):
        index = -i-1
        if i ==0:
            h = output
            z = input_acfun[index]
            delta = deltas0*layers[index]["de_act_fun"](z,h)
        else:
            h = inputs[index+1]
            z = input_acfun[index]
            # print(layers[index]["de_act_fun"](z,h)[1])
            delta = np.dot(delta,layers[index+1]["w"].T)*layers[index]["de_act_fun"](z,h)
        
        deltas.insert(0,delta)
    
    # 利用误差 对每一层的权重进行修成                                        
    for i in range(len(layers)):
        # 计算 dw 与 db
        dw = np.dot(inputs[i].T,deltas[i])
        db = np.sum(deltas[i],axis=0,keepdims=True)
        # 梯度下降
        layers[i]["w"] = layers[i]["w"] - alpha*dw
        layers[i]["b"] = layers[i]["b"] - alpha*db
        
    return layers,loss
    
    
def test_accuracy(datas,labs_true,layers):
    _,_,output = fead_forward(datas,layers)
    lab_det = np.argmax(output,axis=1)
    labs_true = np.argmax(labs_true,axis=1) 
    N_error = np.where(np.abs(labs_true-lab_det)>0)[0].shape[0]
   
    error_rate = N_error/np.shape(datas)[0]
    return error_rate

    
def load_dataset_iris(file_data,N_train):
    # 数据读取
    datas = np.loadtxt(file_data,dtype = np.float, delimiter = ',',usecols=(0,1,2,3))
    labs = np.loadtxt(file_data,dtype = str, delimiter = ',',usecols=(4))
    N,D = np.shape(datas)
    N_test = N-N_train
    unqiue_labs = np.unique(labs).tolist()
    
    dic_str2index={}
    dic_index2str={}
    for i in range(len(unqiue_labs)):
        lab_str = unqiue_labs[i]
        dic_str2index[lab_str] =i
        dic_index2str[i]=lab_str
    
    labs_onehot = np.zeros([N,len(unqiue_labs)])
    for i in range(N):
        labs_onehot[i,dic_str2index[labs[i]]]=1
    
    perm = np.random.permutation(N)
    index_train = perm[:N_train]
    index_test = perm[N_train:]

    data_train = datas[index_train,:]
    lab_train_onehot = labs_onehot[index_train,:]

    data_test = datas[index_test,:]
    lab_test_onehot = labs_onehot[index_test]

    return data_train,lab_train_onehot,data_test,lab_test_onehot,dic_index2str
    
    
    
if __name__=="__main__":
    
    file_data = 'iris.data'

    data_train,lab_train_onehot,data_test,lab_test_onehot,dic_index2str =load_dataset_iris(file_data,100)
    
    N,dim_in = np.shape(data_train)
    # 定义网络结构
    list_num_hidden=[10,5,3]
    list_act_funs =[relu,relu,no_active]
    list_de_act_funs=[de_relu,de_relu,de_no_active]
    
    # 定义损失函数
    loss_fun = loss_CE
    de_loss_fun=de_loss_CE
    
    # loss_fun = loss_L2
    # de_loss_fun=de_loss_L2
    
    layers = bulid_net(dim_in,list_num_hidden,
          list_act_funs,list_de_act_funs)
    
   
    # 进行训练
    n_epoch = 50
    batchsize =4    
    N_batch = N//batchsize
    for i in range(n_epoch):
        # 数据打乱
        rand_index  = np.random.permutation(N).tolist()
        # 每个batch 更新一下weight
        loss_sum =0
        for j in range(N_batch):
            index = rand_index[j*batchsize:(j+1)*batchsize]
            batch_datas = data_train[index]
            batch_labs = lab_train_onehot[index]
            layers,loss = updata_wb(batch_datas,batch_labs,layers,loss_fun,de_loss_fun,alpha=0.01)
            loss_sum = loss_sum+loss
            
        error = test_accuracy(data_train,lab_train_onehot,layers)
        print("epoch %d  error  %.2f%%  loss_all %.2f"%(i,error*100,loss_sum))
    
    #进行测试
    error = test_accuracy(data_test,lab_test_onehot,layers)
    print(error*100)    

<ipython-input-1-dee64462ff95>:150: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  datas = np.loadtxt(file_data,dtype = np.float, delimiter = ',',usecols=(0,1,2,3))


epoch 0  error  66.00%  loss_all 110.19
epoch 1  error  66.00%  loss_all 110.19
epoch 2  error  66.00%  loss_all 110.19
epoch 3  error  66.00%  loss_all 110.11
epoch 4  error  66.00%  loss_all 110.22
epoch 5  error  66.00%  loss_all 110.19
epoch 6  error  66.00%  loss_all 110.17
epoch 7  error  66.00%  loss_all 110.15
epoch 8  error  66.00%  loss_all 110.18
epoch 9  error  66.00%  loss_all 110.22
epoch 10  error  66.00%  loss_all 110.23
epoch 11  error  66.00%  loss_all 110.13
epoch 12  error  66.00%  loss_all 110.14
epoch 13  error  66.00%  loss_all 110.21
epoch 14  error  66.00%  loss_all 110.11
epoch 15  error  66.00%  loss_all 110.21
epoch 16  error  66.00%  loss_all 110.11
epoch 17  error  66.00%  loss_all 110.10
epoch 18  error  66.00%  loss_all 110.25
epoch 19  error  66.00%  loss_all 110.15
epoch 20  error  66.00%  loss_all 110.18
epoch 21  error  66.00%  loss_all 110.12
epoch 22  error  66.00%  loss_all 110.25
epoch 23  error  66.00%  loss_all 110.22
epoch 24  error  66.00%  l

In [5]:
##coding: utf-8
import numpy as np
from math import sqrt
from sklearn.datasets import load_iris
from numpy import *



#BP神经网络模型的训练
def bp_train(feature, label, n_hidden, maxCycle, alpha, n_output):
    """计算隐含层的输入
    input:  feature(mat): 特征
            label(mat):  标签
            n_hidden(int): 隐含层的节点数
            maxCycle(int):  最大的迭代次数
            alpha(float):  学习率
            n_output(int):  输出层的节点数
    output:  w0(mat):  输入层到隐含层之间的的权重
             b0(mat):  输入层到隐含层之间的偏置
             w1(mat):  隐含层到输出层之间的权重
             b1(mat):  隐含层到输出层之间的偏置
    """
    m, n = np.shape(feature)
    #1. 随机初始化参数（权重，偏置， 网络层结构， 激活函数）
    w0 = np.mat(np.random.rand(n, n_hidden))
    w0 = w0 * (8.0 * sqrt(6) / sqrt(n + n_hidden)) -\
         np.mat(np.ones((n, n_hidden))) * (4.0 * sqrt(6) / sqrt(n + n_hidden))
    b0 = np.mat(np.random.rand(1, n_hidden))
    b0 = b0 * (8.0 * sqrt(6) / sqrt(n + n_hidden)) -\
         np.mat(np.ones((1, n_hidden))) * (4.0 * sqrt(6) / sqrt(n + n_hidden))
    w1 = np.mat(np.random.rand(n_hidden, n_output))
    w1 = w1 * (8.0 * sqrt(6) / sqrt(n_hidden + n_output)) -\
         np.mat(np.ones((n_hidden, n_output))) * (4.0 * sqrt(6) / sqrt(n_hidden + n_output))
    b1 = np.mat(np.random.rand(1, n_output))
    b1 = b1 * (8.0 * sqrt(6) / sqrt(n_hidden + n_output)) -\
         np.mat(np.ones((1, n_output))) * (4.0 * sqrt(6) / sqrt(n_hidden + n_output))

    #2. 训练
    i = 0
    while i <= maxCycle:
        #信号正向传播
        #计算隐含层的输入
        hidden_input = hidden_in(feature, w0, b0)
        #计算隐含层的输出
        hidden_output = hidden_out(hidden_input)
        #计算输出层的输入
        output_in = predict_in(hidden_output, w1, b1)
        #计算输出层的输出
        output_out = predict_out(output_in)

        #误差的反向传播
        #隐含层到输出层之间的残差
        delta_output = -np.multiply((label - output_out), partial_sig(output_in))
        #输入层到隐含层之间的残差
        delta_hidden = np.multiply((delta_output * w1.T), partial_sig(hidden_input))

        #修正权重和偏置
        w1 = w1 - alpha * (hidden_output.T * delta_output)
        b1 = b1 - alpha * np.sum(delta_output, axis=0) * (1.0 / m)
        w0 = w0 - alpha * (feature.T * delta_hidden)
        b0 = b0 - alpha * np.sum(delta_hidden, axis=0) * (1.0 / m)
        if i % 100 == 0:
            print("\t------iter: ", i, ", cost: ", (1.0/2) * get_cost(get_predict(feature, w0, w1, b0, b1) - label))
        i += 1
    return w0, w1, b0, b1

#计算隐含层的输入的hidden_in函数
def hidden_in(feature, w0, b0):
    """隐含层的输入
    input:  feature(mat): 特征
            w0(mat): 输入层到隐含层之间的权重
            b0(mat): 输入层到隐含层之间的偏置
    output:  hidden_in(mat): 隐含层的输入
    """
    m = np.shape(feature)[0]
    hidden_in = feature * w0
    for i in range(m):
        hidden_in[i, ] += b0
    return hidden_in

#计算隐含层的输出的hidden_out函数
def hidden_out(hidden_in):
    """隐含层的输出
    input:  hidden_in(mat): 隐含层的输入
    output: hidden_output(mat): 隐含层的输出
    """
    hidden_output = sig(hidden_in)
    return hidden_output

#计算输出层的输入的predict_in函数
def predict_in(hidden_out, w1, b1):
    """输出层的输入
    input:  hidden_out(mat): 隐含层的输出
            w1(mat): 隐含层到输出层之间的权重
            b1(mat): 隐含层到输出层之间的偏置
    output:  predict_in(mat): 输出层的输入
    """
    m = np.shape(hidden_out)[0]
    predict_in = hidden_out * w1
    for i in range(m):
        predict_in[i, ] += b1
    return predict_in

#计算输出层的输出的predict_out函数
def predict_out(predict_in):
    """输出层的输出
    input:  predict_in(mat): 输出层的输入
    output:  result(mat): 输出层的输出
    """
    result = sig(predict_in)
    return result

#Sigmoid函数
def sig(x):
    """Sigmoid激活函数
    input:  x(mat/float): 自变量（矩阵或者任意实数）
    output: Sigmoid值（mat/float）: Sigmoid函数的值
    """
    return 1.0 / (1 + np.exp(-x))

#partial_sig函数
def partial_sig(x):
    m, n = np.shape(x)
    out = np.mat(np.zeros((m, n)))
    for i in range(m):
        for j in range(n):
            out[i, j] = sig(x[i, j]) * (1 - sig(x[i, j]))
    return out

#计算损失函数值的get_cost函数
def get_cost(cost):
    m, n = np.shape(cost)
    cost_sum = 0.0
    for i in range(m):
        for j in range(n):
            cost_sum += cost[i, j] * cost[i, j]
    return cost_sum / m

#计算错误率的err_rate函数
def err_rate(label, pre):
    m = np.shape(label)[0]
    err = 0.0
    for i in range(m):
        if label[i, 0] != pre[i, 0]:
            err += 1
    rate = err / m
    return rate


# #导入数据的load_data函数
# def load_data(filename):
#     """导入训练数据
#     input：  filename(string): 文件名
#     output:  feature_name(mat): 特征
#             label_data(mat): 标签
#             n_class(int): 类别的个数
#     """
#     #1.获取特征
#     f = open(filename) #也可以用上下文管理器进行管理文件的打开和关闭 with open(filename) as f:这样就不用f.close进行关闭
#     feature_data = []
#     label_tmp = []
#     for line in f.readlines():
#         feature_tmp = []
#         lines = line.strip().split("\t")
#         for i in range(len(lines) - 1):
#             feature_tmp.append(float(lines[i]))
#         label_tmp.append(int(lines[-1]))
#         feature_data.append(feature_tmp)
#    # f.close()
#
#     #2.获取标签
#     m = len(label_tmp)
#     n_class = len(set(label_tmp))
#     label_data = np.mat(np.zeros((m , n_class)))
#     for i in range(m):
#         label_data[i, label_tmp[i]] = 1
#     return np.mat(feature_data), label_data, n_class

def load_data():
    dataset_iris = load_iris()
    data_iris = dataset_iris['data']
    label_tmp = dataset_iris['target']
    m = len(label_tmp)
    n_class = len(set(label_tmp))
    print(label_tmp)
    label_iris = np.mat(np.zeros((m, n_class)))
    for i in range(m):
        label_iris[i, label_tmp[i]] = 1
    print(label_iris)
    return data_iris, label_iris, n_class


#保存bp模型的save_model函数
def save_model(w0, w1, b0, b1):
    def write_file(filename, source):
        f = open(filename, "w")
        m, n = np.shape(source)
        for i in range(m):
            tmp = []
            for j in range(n):
                tmp.append(str(source[i, j]))
            f.write("\t".join(tmp) + "\n")
        f.close()
    write_file("weight_w0", w0)
    write_file("weight_w1", w1)
    write_file("weight_b0", b0)
    write_file("weight_b1", b1)

#对测试样本进行预测的get_predict函数
def get_predict(feature, w0, w1, b0, b1):
    return predict_out(predict_in(hidden_out(hidden_in(feature, w0, b0)), w1, b1))

import copy 
import mglearn
import matplotlib.pyplot as plt 

#训练BP模型的主函数
if __name__ == "__main__":
    #1.导入数据
    print("----------1. load Data-------------")
    feature, label, n_class = load_data()

    plt.rcParams['figure.figsize'] = (16.0, 16.0)
    temp = copy.deepcopy(corr_names[arg_indexs[0]])
    temp = temp[len(temp)-5:-1]
    temp.append(corr_matrix.columns[arg_indexs[0]])
    grr = pd.plotting.scatter_matrix(feature[temp],marker='o',c = train_label.iloc[:,2],hist_kwds={'bins':20})#,cmap=mglearn.cm3)
    plt.savefig("imgs/第1题强相关变量之间的散点图1.png",dpi=300)
    #2.训练模型
    print("----------2. training--------------")
    w0, w1, b0, b1 = bp_train(feature, label, 30, 1000, 0.01, n_class)
    #3.保存模型
    print("----------3. save model-------------")
    save_model(w0, w1, b0, b1)
    #4.得到最终的预测结果
    print("----------4. get prediction----------")
    result = get_predict(feature, w0, w1, b0, b1)
    print(result)
    print("训练准确性为： ", (1 - err_rate(np.argmax(label, axis=1), np.argmax(result, axis=1))))

----------1. load Data-------------
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]
[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.

In [ ]:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   ‘